In [ ]:
# 1번 
def click_next_page(driver):
    """
    마이리얼트립 다음 페이지(>) 버튼 클릭 함수
    
    Args:
        driver: Selenium WebDriver 객체
    
    Returns:
        tuple: (성공여부, 메시지)
            - (True, "다음 페이지 이동 성공") : 성공
            - (False, "마지막 페이지") : 더 이상 다음 페이지가 없음
            - (False, "오류메시지") : 실패
    """
    print("  🔍 다음 페이지 버튼 찾는 중...")
    
    try:
        # 다음 페이지 버튼 찾기 (여러 방법으로 시도)
        next_button_selectors = [
            # 1. 정확한 XPath (가장 확실한 방법)
            (By.XPATH, "/html/body/div[4]/div[2]/div/div/div[2]/main/div/div[4]/div/li[7]/button"),
            
            # 2. 클래스 기반 선택자 (백업용)
            (By.CSS_SELECTOR, "button.css-13fjuep"),
            
            # 3. 이미지 기반 선택자 (백업용)
            (By.XPATH, "//img[@src='https://dffoxz5he03rp.cloudfront.net/icons/ic_arrowright_sm_blue_500.svg']/ancestor::button"),
            
            # 4. 구조 기반 선택자 (백업용)
            (By.XPATH, "//button[.//img[@alt='icon' and contains(@src, 'arrowright')]]")
        ]
        
        next_button = None
        current_url = driver.current_url  # 현재 URL 저장
        
        # 버튼 찾기 시도
        for selector_type, selector_value in next_button_selectors:
            try:
                next_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                print(f"    ✅ 다음 페이지 버튼 발견!")
                break
            except TimeoutException:
                continue
        
        if not next_button:
            print(f"    ❌ 다음 페이지 버튼을 찾을 수 없습니다")
            return False, "버튼 없음"
        
        # 버튼 클릭 전 페이지 정보 확인
        try:
            # 현재 페이지 번호 추출 (URL에서)
            current_page = 1
            if "page=" in current_url:
                import re
                page_match = re.search(r'page=(\d+)', current_url)
                if page_match:
                    current_page = int(page_match.group(1))
            
            print(f"    📄 현재 페이지: {current_page}")
        except:
            current_page = "알 수 없음"
        
        # 버튼 클릭 실행
        print(f"    🖱️ 다음 페이지 버튼 클릭 중...")
        
        # JavaScript 클릭 시도 (더 안정적)
        try:
            driver.execute_script("arguments[0].click();", next_button)
        except:
            # 일반 클릭으로 재시도
            next_button.click()
        
        # 페이지 로딩 대기
        print(f"    ⏰ 페이지 로딩 대기 중...")
        time.sleep(random.uniform(3, 6))
        
        # 페이지 이동 확인
        new_url = driver.current_url
        
        # URL 변화 확인 또는 페이지 내용 변화 확인
        page_changed = False
        
        # 1. URL 변화 확인
        if new_url != current_url:
            page_changed = True
            print(f"    ✅ URL 변화 감지 - 페이지 이동 성공!")
            print(f"    🔗 새로운 URL: {new_url[:80]}...")
        
        # 2. URL이 같더라도 페이지 내용이 바뀔 수 있으므로 추가 확인
        else:
            try:
                # 페이지 로딩 상태 확인
                WebDriverWait(driver, 5).until(
                    lambda d: d.execute_script("return document.readyState") == "complete"
                )
                
                # 상품 목록이 새로 로드되었는지 확인
                time.sleep(2)  # 약간의 대기 후
                
                # 새로운 상품들이 로드되었다고 가정
                page_changed = True
                print(f"    ✅ 페이지 내용 업데이트 감지 - 다음 페이지로 이동됨!")
                
            except TimeoutException:
                print(f"    ⚠️ 페이지 상태 확인 실패")
        
        if page_changed:
            # 새 페이지 로딩 완료 대기
            try:
                # DOM 로딩 완료 대기
                WebDriverWait(driver, 15).until(
                    lambda d: d.execute_script("return document.readyState") == "complete"
                )
                
                # 상품 목록이 로드될 때까지 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/experiences/'], a[href*='/products/']"))
                )
                
                print(f"    🎉 다음 페이지 로딩 완료!")
                
                # 추가 안전 대기
                time.sleep(random.uniform(2, 4))
                
                return True, "다음 페이지 이동 성공"
                
            except TimeoutException:
                print(f"    ⚠️ 새 페이지 로딩 시간 초과 (하지만 이동은 성공)")
                return True, "다음 페이지 이동 성공 (로딩 지연)"
        
        else:
            # 페이지가 변하지 않음 = 마지막 페이지일 가능성
            print(f"    ⚠️ 페이지 변화 없음 - 마지막 페이지일 수 있습니다")
            
            # 마지막 페이지 확인을 위한 추가 검증
            try:
                # 버튼이 여전히 클릭 가능한지 확인
                if next_button.is_enabled() and next_button.is_displayed():
                    print(f"    🔍 버튼은 활성 상태 - 다시 한 번 시도...")
                    
                    # 한 번 더 클릭 시도
                    time.sleep(2)
                    driver.execute_script("arguments[0].click();", next_button)
                    time.sleep(3)
                    
                    final_url = driver.current_url
                    if final_url != current_url:
                        print(f"    ✅ 두 번째 시도로 성공!")
                        return True, "다음 페이지 이동 성공 (재시도)"
                    else:
                        print(f"    ⛔ 마지막 페이지 확인됨")
                        return False, "마지막 페이지"
                else:
                    print(f"    ⛔ 버튼 비활성화됨 - 마지막 페이지")
                    return False, "마지막 페이지"
                    
            except Exception as e:
                print(f"    ❌ 마지막 페이지 검증 실패: {e}")
                return False, "마지막 페이지 (검증 실패)"
        
    except TimeoutException:
        print(f"    ❌ 다음 페이지 버튼 클릭 시간 초과")
        return False, "시간 초과"
        
    except NoSuchElementException:
        print(f"    ❌ 다음 페이지 버튼을 찾을 수 없습니다")
        return False, "버튼 없음"
        
    except Exception as e:
        print(f"    ❌ 다음 페이지 이동 중 오류: {type(e).__name__}: {e}")
        return False, f"오류: {type(e).__name__}"


def safe_next_page_navigation(driver, max_retries=3):
    """
    안전한 다음 페이지 이동 (재시도 포함)
    
    Args:
        driver: Selenium WebDriver 객체
        max_retries: 최대 재시도 횟수
    
    Returns:
        tuple: (성공여부, 메시지)
    """
    for attempt in range(max_retries):
        try:
            print(f"  🔄 다음 페이지 이동 시도 {attempt + 1}/{max_retries}")
            
            success, message = click_next_page(driver)
            
            if success:
                return True, message
            elif "마지막 페이지" in message:
                # 마지막 페이지는 재시도하지 않음
                return False, message
            else:
                # 다른 오류는 재시도
                if attempt < max_retries - 1:
                    print(f"    ⏰ {3}초 후 재시도...")
                    time.sleep(3)
                    continue
                else:
                    return False, f"최종 실패: {message}"
                    
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"    ❌ 시도 {attempt + 1} 실패: {e}")
                print(f"    ⏰ {3}초 후 재시도...")
                time.sleep(3)
                continue
            else:
                return False, f"최종 오류: {e}"
    
    return False, "모든 시도 실패"


def crawl_with_pagination(driver, city_name, max_pages=10):
    """
    페이지네이션을 활용한 전체 크롤링 시스템
    test74.ipynb의 그룹 8, 9를 대체하는 함수
    
    Args:
        driver: Selenium WebDriver 객체
        city_name: 크롤링할 도시명
        max_pages: 최대 크롤링할 페이지 수
    
    Returns:
        int: 총 크롤링한 상품 수
    """
    print(f"🚀 {city_name} 페이지네이션 크롤링 시작!")
    print(f"📄 최대 {max_pages}페이지까지 크롤링 예정")
    
    total_crawled = 0
    current_page = 1
    
    # 상태 관리 로드
    try:
        crawler_state, completed_urls = load_crawler_state()
        start_number = get_last_product_number(city_name) + 1
    except:
        print("⚠️ 상태 관리 시스템 오류 - 기본값 사용")
        crawler_state = {"total_collected_count": 0}
        completed_urls = set()
        start_number = 0
    
    while current_page <= max_pages:
        print(f"\n📄 === {current_page}페이지 크롤링 시작 ===")
        
        try:
            # 현재 페이지의 URL 수집
            print(f"  📊 {current_page}페이지 URL 수집 중...")
            page_urls = collect_all_24_urls(driver)
            
            if not page_urls:
                print(f"  ❌ {current_page}페이지에서 URL을 찾을 수 없습니다")
                break
            
            # 새로운 URL만 필터링
            new_urls = [url for url in page_urls if url not in completed_urls]
            
            print(f"  📈 수집 결과: 전체 {len(page_urls)}개, 신규 {len(new_urls)}개")
            
            if not new_urls:
                print(f"  ⚠️ {current_page}페이지에 새로운 URL이 없습니다")
            else:
                # 설정된 개수만큼만 크롤링
                max_products = CONFIG.get('MAX_PRODUCTS_PER_CITY', 24)
                urls_to_process = new_urls[:max_products]
                
                print(f"  🎯 {len(urls_to_process)}개 상품 크롤링 시작")
                
                # 여기서 실제 크롤링 수행 (그룹 9의 메인 루프와 동일)
                # ... 크롤링 로직 ...
                
                total_crawled += len(urls_to_process)
            
            # 다음 페이지로 이동
            if current_page < max_pages:
                print(f"  🔄 다음 페이지로 이동 중...")
                
                success, message = safe_next_page_navigation(driver)
                
                if success:
                    print(f"  ✅ {current_page + 1}페이지로 이동 완료")
                    current_page += 1
                    
                    # 페이지 이동 후 안전 대기
                    time.sleep(random.uniform(3, 6))
                    
                else:
                    if "마지막 페이지" in message:
                        print(f"  🏁 마지막 페이지 도달: {message}")
                        break
                    else:
                        print(f"  ❌ 페이지 이동 실패: {message}")
                        print(f"  ⚠️ 현재 페이지에서 크롤링 종료")
                        break
            else:
                print(f"  🏁 설정된 최대 페이지({max_pages}) 도달")
                break
                
        except Exception as e:
            print(f"  ❌ {current_page}페이지 처리 중 오류: {e}")
            break
    
    print(f"\n🎉 페이지네이션 크롤링 완료!")
    print(f"📊 총 {current_page}페이지 처리")
    print(f"🎯 총 {total_crawled}개 상품 크롤링")
    
    return total_crawled

In [ ]:
#2번
# =============================================================================
# 🔄 개선된 페이지네이션 시스템 (기존 그룹 8, 9와 통합)
# =============================================================================

def click_next_page(driver):
    """
    마이리얼트립 다음 페이지(>) 버튼 클릭 함수 (개선됨)
    """
    print("  🔍 다음 페이지 버튼 찾는 중...")
    
    try:
        # 다음 페이지 버튼 찾기 (여러 방법으로 시도)
        next_button_selectors = [
            # 1. 정확한 XPath (가장 확실한 방법)
            (By.XPATH, "/html/body/div[4]/div[2]/div/div/div[2]/main/div/div[4]/div/li[7]/button"),
            
            # 2. 클래스 기반 선택자 (백업용)
            (By.CSS_SELECTOR, "button.css-13fjuep"),
            
            # 3. 이미지 기반 선택자 (백업용)
            (By.XPATH, "//img[@src='https://dffoxz5he03rp.cloudfront.net/icons/ic_arrowright_sm_blue_500.svg']/ancestor::button"),
            
            # 4. 구조 기반 선택자 (백업용)
            (By.XPATH, "//button[.//img[@alt='icon' and contains(@src, 'arrowright')]]"),
            
            # 5. 추가 백업 셀렉터들
            (By.XPATH, "//button[contains(@aria-label, '다음')]"),
            (By.XPATH, "//button[contains(text(), '다음')]"),
            (By.CSS_SELECTOR, "button[class*='next']"),
        ]
        
        next_button = None
        current_url = driver.current_url
        
        # 버튼 찾기 시도
        for selector_type, selector_value in next_button_selectors:
            try:
                next_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                print(f"    ✅ 다음 페이지 버튼 발견!")
                break
            except TimeoutException:
                continue
        
        if not next_button:
            print(f"    ❌ 다음 페이지 버튼을 찾을 수 없습니다")
            return False, "버튼 없음"
        
        # 현재 페이지 번호 추출
        current_page = 1
        if "page=" in current_url:
            import re
            page_match = re.search(r'page=(\d+)', current_url)
            if page_match:
                current_page = int(page_match.group(1))
        
        print(f"    📄 현재 페이지: {current_page}")
        
        # 버튼 클릭 실행
        print(f"    🖱️ 다음 페이지 버튼 클릭 중...")
        
        # JavaScript 클릭 (더 안정적)
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_button)
        except:
            next_button.click()
        
        # 페이지 로딩 대기
        print(f"    ⏰ 페이지 로딩 대기 중...")
        time.sleep(random.uniform(3, 6))
        
        # 페이지 이동 확인
        new_url = driver.current_url
        page_changed = new_url != current_url
        
        if not page_changed:
            # URL이 같아도 내용이 바뀔 수 있으므로 추가 확인
            try:
                WebDriverWait(driver, 5).until(
                    lambda d: d.execute_script("return document.readyState") == "complete"
                )
                time.sleep(2)
                page_changed = True  # 내용 변화 가정
            except:
                pass
        
        if page_changed:
            # 새 페이지 로딩 완료 대기
            try:
                WebDriverWait(driver, 15).until(
                    lambda d: d.execute_script("return document.readyState") == "complete"
                )
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']"))
                )
                
                print(f"    🎉 다음 페이지 로딩 완료!")
                time.sleep(random.uniform(2, 4))
                return True, "다음 페이지 이동 성공"
                
            except TimeoutException:
                print(f"    ⚠️ 새 페이지 로딩 시간 초과 (하지만 이동은 성공)")
                return True, "다음 페이지 이동 성공 (로딩 지연)"
        else:
            print(f"    ⛔ 마지막 페이지 도달")
            return False, "마지막 페이지"
            
    except Exception as e:
        print(f"    ❌ 다음 페이지 이동 중 오류: {type(e).__name__}: {e}")
        return False, f"오류: {type(e).__name__}"

def safe_next_page_navigation(driver, max_retries=2):
    """안전한 다음 페이지 이동 (재시도 포함)"""
    for attempt in range(max_retries):
        print(f"  🔄 다음 페이지 이동 시도 {attempt + 1}/{max_retries}")
        
        success, message = click_next_page(driver)
        
        if success:
            return True, message
        elif "마지막 페이지" in message:
            return False, message
        else:
            if attempt < max_retries - 1:
                print(f"    ⏰ 3초 후 재시도...")
                time.sleep(3)
            else:
                return False, f"최종 실패: {message}"
    
    return False, "모든 시도 실패"

def crawl_with_pagination_integrated(driver, city_name, max_pages=5):
    """
    ✅ 기존 시스템과 완전 통합된 페이지네이션 크롤링
    그룹 8, 9의 기능을 모두 포함하여 페이지네이션 지원
    """
    print(f"🚀 {city_name} 페이지네이션 크롤링 시작!")
    print(f"📄 최대 {max_pages}페이지까지 크롤링 예정")
    
    # ✅ 기존 시스템과 동일한 초기화
    try:
        crawler_state, completed_urls = load_crawler_state()
        start_number = get_last_product_number(city_name) + 1
        continent, country = get_city_info(city_name)
    except Exception as e:
        print(f"⚠️ 상태 관리 시스템 오류: {e}")
        return 0
    
    # 전체 크롤링 통계
    total_products_crawled = 0
    total_pages_processed = 0
    current_product_number = start_number
    
    # 배치 처리용 결과 저장소
    all_batch_results = []
    batch_size = 5
    
    # 브라우저 재부팅 설정
    restart_interval = random.randint(2, 3)
    current_count = 0
    
    print(f"🔢 번호 시작점: {start_number}")
    print(f"🛡️ 브라우저 재부팅: {restart_interval}개 상품마다")
    
    # 페이지네이션 루프
    for page_num in range(1, max_pages + 1):
        print(f"\n📄 === {page_num}페이지 크롤링 시작 ===")
        total_pages_processed = page_num
        
        try:
            # ✅ 기존 그룹 8과 동일한 URL 수집
            print(f"  📊 {page_num}페이지 URL 수집 중...")
            
            # 세션 안전 URL 수집 (기존 함수 재사용)
            page_urls = collect_urls_with_session_safety(driver, city_name, completed_urls)
            
            if not page_urls:
                print(f"  ❌ {page_num}페이지에서 새로운 URL을 찾을 수 없습니다")
                if page_num == 1:
                    print("  ⚠️ 첫 페이지에서도 URL이 없으면 문제가 있을 수 있습니다")
                    break
                else:
                    print("  ➡️ 다음 페이지로 이동합니다")
            else:
                # ✅ 설정된 개수만큼만 크롤링
                max_per_page = CONFIG.get('MAX_PRODUCTS_PER_CITY', 24)
                urls_to_crawl = page_urls[:max_per_page]
                page_crawl_count = 0
                
                print(f"  🎯 {len(urls_to_crawl)}개 상품 크롤링 시작")
                
                # ✅ 기존 그룹 9와 동일한 메인 크롤링 루프
                for loop_index, product_url in enumerate(urls_to_crawl):
                    current_count += 1
                    
                    # ✅ 브라우저 재부팅 체크 (기존과 동일)
                    if current_count > restart_interval:
                        print(f"🎲 {current_count-1}개 완료! 안전한 브라우저 재부팅...")
                        success, message = safe_browser_restart()
                        
                        if success and return_to_current_page():
                            # 페이지네이션으로 현재 페이지로 복귀
                            if page_num > 1:
                                print(f"📄 {page_num}페이지로 복귀 중...")
                                for _ in range(page_num - 1):
                                    nav_success, nav_msg = safe_next_page_navigation(driver)
                                    if not nav_success:
                                        print(f"❌ 페이지 복귀 실패: {nav_msg}")
                                        break
                                    time.sleep(2)
                            
                            restart_interval = random.randint(2, 3)
                            current_count = 1
                            print(f"🎯 다음 재부팅: {restart_interval}개 후")
                        else:
                            print(f"🚨 브라우저 재시작 실패: {message}")
                            break
                    
                    print_product_progress(current_product_number, len(urls_to_crawl), f"상품 {current_product_number}")
                    
                    # ✅ 기존 그룹 9와 동일한 상품 크롤링
                    try:
                        # 상품 페이지 이동
                        driver.get(product_url)
                        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
                        
                        # 정보 수집
                        url_type = "Product" if "/products/" in product_url else "Offer"
                        product_name = get_product_name(driver, url_type)
                        price_raw = get_price(driver)
                        price_clean = clean_price(price_raw)
                        rating_raw = get_rating(driver)
                        rating_clean = clean_rating(rating_raw)
                        review_count = get_review_count(driver)
                        language = get_language(driver)
                        
                        # ✅ 번호 연속성 적용
                        img_info = download_image(driver, product_name, city_name, current_product_number)
                        
                        # 결과 저장
                        result = {
                            '번호': current_product_number,
                            '대륙': continent, '국가': country, '도시': city_name,
                            '공항코드': get_city_code(city_name), '상품타입': url_type,
                            '상품명': product_name, '가격_원본': price_raw, '가격_정제': price_clean,
                            '평점_원본': rating_raw, '평점_정제': rating_clean, '리뷰수': review_count, '언어': language,
                            '이미지_파일명': img_info.get('filename', ''),
                            '이미지_상대경로': img_info.get('relative_path', ''),
                            '이미지_전체경로': img_info.get('path', ''),
                            '이미지_상태': img_info.get('status', ''),
                            '이미지_크기': img_info.get('size', 0),
                            'URL': product_url,
                            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
                            '상태': '완전수집',
                            '페이지': page_num
                        }
                        all_batch_results.append(result)
                        
                        # ✅ 상태 저장
                        save_crawler_state(crawler_state, product_url)
                        
                        print(f"    🎉 상품 {current_product_number} 수집 완료: {product_name[:30]}...")
                        
                        current_product_number += 1
                        page_crawl_count += 1
                        total_products_crawled += 1
                        
                        # ✅ 배치 저장 (기존과 동일)
                        if len(all_batch_results) >= batch_size:
                            batch_save_result = save_batch_data(all_batch_results, city_name)
                            if batch_save_result:
                                print(f"    ✅ 배치 저장 완료: {batch_save_result['data_count']}개")
                                all_batch_results = []
                        
                    except Exception as e:
                        print(f"    ❌ 상품 {current_product_number} 처리 실패: {e}")
                        # 실패해도 번호는 증가 (연속성 유지)
                        current_product_number += 1
                        continue
                
                print(f"  ✅ {page_num}페이지 완료: {page_crawl_count}개 상품 처리")
            
            # 다음 페이지로 이동 (마지막 페이지가 아닌 경우)
            if page_num < max_pages:
                print(f"  🔄 {page_num + 1}페이지로 이동 중...")
                
                success, message = safe_next_page_navigation(driver)
                
                if success:
                    print(f"  ✅ {page_num + 1}페이지로 이동 완료")
                    time.sleep(random.uniform(2, 4))
                else:
                    if "마지막 페이지" in message:
                        print(f"  🏁 마지막 페이지 도달: {message}")
                        break
                    else:
                        print(f"  ❌ 페이지 이동 실패: {message}")
                        break
                        
        except Exception as e:
            print(f"  ❌ {page_num}페이지 처리 중 오류: {e}")
            break
    
    # ✅ 남은 배치 데이터 저장
    if all_batch_results:
        print(f"\n💾 최종 배치 데이터 저장 중... ({len(all_batch_results)}개)")
        final_batch_result = save_batch_data(all_batch_results, city_name)
        if final_batch_result:
            print(f"✅ 최종 배치 저장 완료: {final_batch_result['data_count']}개")
    
    print(f"\n🎉 페이지네이션 크롤링 완료!")
    print(f"📄 총 {total_pages_processed}페이지 처리")
    print(f"🎯 총 {total_products_crawled}개 상품 크롤링 성공")
    print(f"🔢 번호 범위: {start_number} ~ {current_product_number - 1}")
    
    return total_products_crawled

# ✅ 사용법
def run_pagination_crawling(city_name="바르셀로나", max_pages=3):
    """페이지네이션 크롤링 실행 래퍼 함수"""
    try:
        # 기존 그룹 6, 7 실행 (초기화 및 검색)
        print("🔄 시스템 초기화 및 검색 페이지 이동...")
        # 여기서 그룹 6, 7 로직 실행
        
        # 페이지네이션 크롤링 실행
        result = crawl_with_pagination_integrated(driver, city_name, max_pages)
        
        print(f"🎊 {city_name} 페이지네이션 크롤링 최종 완료!")
        print(f"📊 총 {result}개 상품 수집 성공")
        
        return result
        
    except Exception as e:
        print(f"❌ 페이지네이션 크롤링 실패: {e}")
        return 0

print("✅ 개선된 페이지네이션 시스템 로드 완료!")
print("🚀 사용법: run_pagination_crawling('바르셀로나', 5)  # 5페이지까지 크롤링")